<a href="https://colab.research.google.com/github/feniix/kinemotion/blob/main/presentation/demos/api_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kinemotion Python API Demo

Interactive demonstration of Kinemotion's Python API for drop jump analysis.

## Setup

Make sure Kinemotion is installed:
```bash
pip install kinemotion
```


In [1]:
!pip install kinemotion

In [2]:
# Import Kinemotion
import json
from pathlib import Path

from kinemotion import process_dropjump_video


/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.7.2 is installed, but it is not compatible with the installed jaxlib version 0.7.1, so it will not be used.
  warnings.warn(


## Demo 1: Single Video Analysis

Process a single drop jump video and extract metrics.


In [3]:
# Process a drop jump video
video_path = "sample_data/IMG_5809.MOV"

print(f"Processing: {video_path}")
print("=" * 60)

metrics = process_dropjump_video(video_path, quality="balanced")

print("\n✅ Analysis Complete!")
print("\n📊 Key Metrics:")

# Handle potential None values
gct_ms = metrics.ground_contact_time * 1000 if metrics.ground_contact_time else 0
ft_ms = metrics.flight_time * 1000 if metrics.flight_time else 0
jh_m = metrics.jump_height if metrics.jump_height else 0

print(f"  Ground Contact Time: {gct_ms:.0f} ms")
print(f"  Flight Time: {ft_ms:.0f} ms")
print(f"  Jump Height: {jh_m:.3f} m")

# Calculate RSI manually (jump height / ground contact time)
gct = metrics.ground_contact_time if metrics.ground_contact_time else 0
rsi = metrics.jump_height / gct if gct and metrics.jump_height else 0
print(f"  Reactive Strength Index: {rsi:.2f}")

Processing: sample_data/IMG_5809.MOV

✅ Analysis Complete!

📊 Key Metrics:
  Ground Contact Time: 324 ms
  Flight Time: 524 ms
  Jump Height: 0.337 m
  Reactive Strength Index: 1.04


## Demo 2: Access All Metrics

Explore the complete metrics dictionary.


In [4]:
# Convert metrics to dictionary for easy viewing
metrics_dict = metrics.to_dict()

print("📋 Complete Metrics:")
print(json.dumps(metrics_dict, indent=2))


📋 Complete Metrics:
{
  "ground_contact_time_ms": 323.75,
  "flight_time_ms": 523.97,
  "jump_height_m": 0.337,
  "jump_height_kinematic_m": 0.337,
  "jump_height_trajectory_normalized": 0.0667,
  "contact_start_frame": 205,
  "contact_end_frame": 224,
  "flight_start_frame": 224,
  "flight_end_frame": 255,
  "peak_height_frame": 231,
  "contact_start_frame_precise": 204.153,
  "contact_end_frame_precise": 223.572,
  "flight_start_frame_precise": 223.572,
  "flight_end_frame_precise": 255.0
}


## Demo 3: Custom Analysis Workflow

Build a custom workflow for team assessment.


In [5]:
# Process multiple athletes and compare
from typing import Any

athletes = [
    "IMG_5809.MOV",
    "IMG_5813.MOV",
    "IMG_5820.MOV",
]

results: list[dict[str, Any]] = []

print("🏃 Processing team...\n")

for athlete_video in athletes:
    video_path = Path("sample_data") / athlete_video

    if not video_path.exists():
        print(f"⚠️  Skipping {athlete_video} (not found)")
        continue

    try:
        metrics = process_dropjump_video(str(video_path))
        # Calculate RSI manually
        gct = metrics.ground_contact_time if metrics.ground_contact_time else 0
        jh = metrics.jump_height if metrics.jump_height else 0
        rsi = jh / gct if (gct and jh) else 0
        results.append({
            "athlete": athlete_video,
            "gct_ms": metrics.ground_contact_time * 1000 if metrics.ground_contact_time else 0,
            "flight_time_ms": metrics.flight_time * 1000 if metrics.flight_time else 0,
            "rsi": rsi,
            "jump_height_m": jh
        })
        print(f"✅ {athlete_video}: RSI = {rsi:.2f}")
    except Exception as e:
        print(f"❌ {athlete_video}: {e}")

print(f"\n✅ Processed {len(results)} athletes")

🏃 Processing team...

✅ IMG_5809.MOV: RSI = 1.04
✅ IMG_5813.MOV: RSI = 0.62
✅ IMG_5820.MOV: RSI = 0.72

✅ Processed 3 athletes


## Demo 4: Data Analysis

Analyze the results and identify top performers.


In [6]:
# Sort by RSI (higher is better)
sorted_results = sorted(results, key=lambda x: x["rsi"], reverse=True)

print("🏆 Team Rankings by RSI:\n")
for i, result in enumerate(sorted_results, 1):
    athlete: str = result["athlete"]
    rsi: float = result["rsi"]
    gct_ms: float = result["gct_ms"]
    jump_height_m: float = result["jump_height_m"]

    print(f"{i}. {athlete}")
    print(f"   RSI: {rsi:.2f}")
    print(f"   GCT: {gct_ms:.0f}ms")
    print(f"   Jump Height: {jump_height_m:.3f}m")
    print()

🏆 Team Rankings by RSI:

1. IMG_5809.MOV
   RSI: 1.04
   GCT: 324ms
   Jump Height: 0.337m

2. IMG_5820.MOV
   RSI: 0.72
   GCT: 400ms
   Jump Height: 0.287m

3. IMG_5813.MOV
   RSI: 0.62
   GCT: 348ms
   Jump Height: 0.217m



## Demo 5: Export Results

Save results for further analysis in Excel, R, or SPSS.


In [7]:
import csv

# Export to CSV
output_file = "team_results.csv"

with open(output_file, 'w', newline='') as f:
    if results:
        writer = csv.DictWriter(f, fieldnames=results[0].keys())
        writer.writeheader()
        writer.writerows(results)

print(f"💾 Results exported to: {output_file}")
print("📊 Ready for statistical analysis!")


💾 Results exported to: team_results.csv
📊 Ready for statistical analysis!


## Key Takeaways

✅ **Simple API** - Just import and call  
✅ **Flexible** - Build custom workflows  
✅ **Extensible** - Integrate with existing systems  
✅ **Export-friendly** - CSV, JSON, or custom formats  

## Questions?

- GitHub: https://github.com/feniix/kinemotion
- PyPI: https://pypi.org/project/kinemotion
- Docs: Full API reference in repository
